# Optimal EV Charging Pricing - Programming Tutorial
*Written by @Gen Li - gen.li@tum.de with the help of ChatGPT*


## 1. Introduction to Jupyter Notebook in VS Code, Anaconda, and Copilot Student Developer License

#### 1.1 Setting Up Jupyter Notebook in VS Code

- Install Visual Studio Code from [here](https://code.visualstudio.com/)
- Install the Python extension for VS Code
- Install Jupyter extension for VS Code

#### 1.2 Setting Up Anaconda Environment

- Download and install Anaconda from [here](https://www.anaconda.com/products/distribution)
- Create a new environment using Anaconda Navigator or command line:
  ```bash
  conda create -n optevcp python=3.9
  conda activate optevcp
  ```
- Install necessary packages in your environment:
  ```bash
  conda install jupyter
  conda install -c conda-forge pyomo
  conda install -c anaconda networkx
  ```

> [Pyomo](https://www.pyomo.org/): Pyomo is a Python-based, open-source optimization modeling language used for defining and solving mathematical models, with a diverse set of optimization capabilities. -> Check the [documentation](https://readthedocs.org/projects/pyomo/downloads/pdf/latest/).

> [NetworkX](https://networkx.org/) is a Python package for the creation, manipulation, and study of complex networks of nodes and edges.
#### 1.3 Get [Cplex](https://www.ibm.com/de-de/products/ilog-cplex-optimization-studio) solver with [education license](https://community.ibm.com/community/user/ai-datascience/blogs/xavier-nodet1/2020/07/09/cplex-free-for-students) and [set it up](https://or.stackexchange.com/questions/4366/downloading-and-setting-up-cplex-for-pyomo) for Pyomo.
Cplex is a high-performance mathematical programming solver for linear programming, mixed integer programming, and other related problems.

#### 1.4 Copilot Student Developer License

- Sign up for the GitHub Student Developer Pack [here](https://techcommunity.microsoft.com/t5/educator-developer-blog/step-by-step-setting-up-github-student-and-github-copilot-as-an/ba-p/3736279)
- Follow instructions to activate your Copilot license in VS Code


In [1]:
# Code block to allow import of modules from parent directory, reducing error when load files.

import os, sys
currentdir = os.path.dirname(os.path.realpath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(currentdir)

## 2. Modelling example: [Facility location problem](https://scipbook.readthedocs.io/en/latest/flp.html)

#### 3.1 Mathematical Formulation

The Capacitated Facility Location Problem (CFLP) can be described as follows:
$$
\begin{align*}
\text{Minimize} \quad & \sum_{i \in I} f_i x_i + \sum_{i \in I} \sum_{j \in J} c_{ij} y_{ij} \\
\text{Subject to} \quad & \sum_{i \in I} y_{ij} = d_j \quad \forall j \in J\\
                        & \sum_{j \in J} y_{ij} \leq M_i x_i \quad \forall i \in I \\
                        & y_{ij} \geq 0 \quad \forall i \in I, j \in J \\
                        & x_i \in \{0, 1\} \quad \forall i \in I
\end{align*}
$$

### Problem Description

Given a set of potential facility locations and a set of customers with specific demands, the objective is to determine the optimal locations to open facilities and the optimal transportation plan from facilities to customers, such that the total cost is minimized while satisfying customer demands and not exceeding facility capacities.

Questions: 
- Which are variables, parameters?
- How to interpret the objective functions and constraints?



##### Decision Variables
- $ x_i $: Binary variable indicating if facility $ i $ is opened.
- $ y_{ij} $: Continuous variable indicating the amount of *goods* transported from facility $ i $ to customer $ j $.

##### Parameters
- $ f_i $: Fixed cost of opening facility $ i $.
- $ c_{ij} $: Transportation cost of serving customer $ j $ from facility $ i $.
- $ d_j $: Demand of customer $ j $.
- $ M_i $: Capacity of facility $ i $.

##### Objective
Minimize the total cost of opening facilities and transporting goods from facilities to customers.
$$
\text{Minimize} \quad \sum_{i} f_i x_i + \sum_{i} \sum_{j} c_{ij} y_{ij}
$$

##### Constraints
1. **Demand Satisfaction**: Each customer’s demand must be satisfied by the facilities.
$$
\sum_{i} y_{ij} = d_j \quad \forall j
$$

2. **Capacity Constraints**: The total goods transported from a facility must not exceed its capacity.
$$
\sum_{j} y_{ij} \leq M_i x_i \quad \forall i
$$

3. **Non-negativity and Binary Constraints**:
$$
y_{ij} \geq 0 \quad \forall i, j
$$
$$
x_i \in \{0, 1\} \quad \forall i
$$





In [10]:
import pyomo.environ as pyo
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


### 3.2 Define and visualize the problem 

In [43]:
# Create aggregated data using pandas DataFrame
data = {
    'facility': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
    'customer': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
    'fixed_cost': [100, 100, 100, 100, 200, 200, 200, 200, 300, 300, 300, 300],
    'capacity': [150, 150, 150, 150, 200, 200, 200, 200, 300, 300, 300, 300],
    'demand': [80, 70, 90, 60, 80, 70, 90, 60, 80, 70, 90, 60],
    'transport_cost': [10, 20, 30, 40, 15, 25, 35, 45, 20, 30, 40, 50]
}
df = pd.DataFrame(data)
df

,facility,customer,fixed_cost,capacity,demand,transport_cost
0,1,1,100,150,80,10
1,1,2,100,150,70,20
2,1,3,100,150,90,30
3,1,4,100,150,60,40
4,2,1,200,200,80,15
5,2,2,200,200,70,25
6,2,3,200,200,90,35
7,2,4,200,200,60,45
8,3,1,300,300,80,20
9,3,2,300,300,70,30


In [70]:
# Function to plot the facility location problem
def plot_facility_location(
    df, 
    plot_transport_edges=True,
    plot_cost_edges=False
    ):
    facilities = df.drop_duplicates(subset=['facility'])[['facility', 'fixed_cost', 'capacity']]
    customers = df.drop_duplicates(subset=['customer'])[['customer', 'demand']]

    # Check if the DataFrame has been updated with model results
    if 'facility_open' not in df.columns:
        df['facility_open'] = 0
        df['transport_amount'] = 0

    fig = go.Figure()

    # Add facilities to the plot
    for idx, row in facilities.iterrows():
        facility_open = df[df['facility'] == row['facility']]['facility_open'].max()
        fig.add_trace(go.Scatter(
            x=[row['facility']],
            y=[0],
            mode='markers+text',
            marker=dict(size=row['capacity'] / 10, color='blue' if facility_open > 0.5 else 'gray'),
            text=f"Facility {int(row['facility'])}<br>Open Cost: {row['fixed_cost']}<br>Capacity: {row['capacity']}",
            textposition="top center",
            name=f"Facility {int(row['facility'])}"
        ))

    # Add customers to the plot
    for idx, row in customers.iterrows():
        demand_satisfied = df[df['customer'] == row['customer']]['transport_amount'].sum() >= row['demand']
        fig.add_trace(go.Scatter(
            x=[row['customer']],
            y=[1],
            mode='markers+text',
            marker=dict(size=row['demand'] / 5, color='red', opacity=1 if demand_satisfied else 0.5),
            text=f"Customer {int(row['customer'])}<br>Demand: {row['demand']}",
            textposition="bottom center",
            name=f"Customer {int(row['customer'])}"
        ))

    # Add edges to represent transportation amounts
    if plot_transport_edges:
        for idx, row in df.iterrows():
            if row['transport_amount'] > 0:
                fig.add_trace(go.Scatter(
                    x=[row['facility'], row['customer']],
                    y=[0, 1],
                    mode='lines',
                    line=dict(width=row['transport_amount'] / 10, color='yellow'),
                    text=f"Amount: {row['transport_amount']}",
                    hoverinfo='text',
                    showlegend=False
                ))

    # Add cost edges to represent transportation costs
    if plot_cost_edges:
        for idx, row in df.iterrows():
            fig.add_trace(go.Scatter(
                x=[row['facility'], row['customer']],
                y=[0, 1],
                mode='lines',
                line=dict(width=row['transport_cost'] / 20, color='green'),
                text=f"Cost: {row['transport_cost']}",
                hoverinfo='text',
                showlegend=False
            ))

    # Update layout
    fig.update_layout(
        title="Capacitated Facility Location Problem",
        xaxis_title="Node",
        yaxis_title="Type",
        xaxis=dict(
            showgrid=False,
            gridwidth=1,
            gridcolor='white',
            linecolor='black'
        ),
        yaxis=dict(
            tickvals=[0, 1],
            ticktext=['Facilities', 'Customers'],
            showgrid=False,
            gridwidth=1,
            gridcolor='white',
            linecolor='black'
        ),
        showlegend=False,
        # plot_bgcolor='white',
        # paper_bgcolor='white',
    )

    fig.show()


In [71]:

# Plot the initial data
plot_facility_location(df, plot_cost_edges=True, plot_transport_edges=True)

# After solving the optimization model, you can call the function again with the updated data
# For example:
# plot_facility_location(updated_df, plot_edges=True)

### 3.3 Build the Pyomo model

In [20]:
# Initialize a Pyomo concrete model
model = pyo.ConcreteModel()

# Define sets
model.I = pyo.Set(initialize=df['facility'].unique())
model.J = pyo.Set(initialize=df['customer'].unique())

# Define parameters
fixed_costs = df.drop_duplicates(subset=['facility']).set_index('facility')['fixed_cost'].to_dict()
capacities = df.drop_duplicates(subset=['facility']).set_index('facility')['capacity'].to_dict()
demands = df.drop_duplicates(subset=['customer']).set_index('customer')['demand'].to_dict()
transport_costs = df.set_index(['facility', 'customer'])['transport_cost'].to_dict()

model.f = pyo.Param(model.I, initialize=fixed_costs)
model.M = pyo.Param(model.I, initialize=capacities)
model.d = pyo.Param(model.J, initialize=demands)
model.c = pyo.Param(model.I, model.J, initialize=transport_costs)

# Define decision variables
model.x = pyo.Var(model.I, within=pyo.Binary)
model.y = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)

# Define objective function
def objective_rule(model):
    return sum(model.f[i] * model.x[i] for i in model.I) + sum(model.c[i, j] * model.y[i, j] for i in model.I for j in model.J)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
def demand_constraint_rule(model, j):
    return sum(model.y[i, j] for i in model.I) == model.d[j]
model.demand_constraint = pyo.Constraint(model.J, rule=demand_constraint_rule)

def capacity_constraint_rule(model, i):
    return sum(model.y[i, j] for j in model.J) <= model.M[i] * model.x[i]
model.capacity_constraint = pyo.Constraint(model.I, rule=capacity_constraint_rule)


In [22]:

# Solve the model using Cplex
solver = pyo.SolverFactory('cplex')
solver.solve(model, tee=False)

# Display results
print("Optimal Facility Locations and Transportation Plan")
print("Facilities to Open:")
for i in model.I:
    if pyo.value(model.x[i]) > 0.5:
        print(f"Facility {i} is opened.")

print("\nTransportation Plan:")
for i in model.I:
    for j in model.J:
        if pyo.value(model.y[i, j]) > 0:
            print(f"Transport {pyo.value(model.y[i, j])} units from Facility {i} to Customer {j}.")

Optimal Facility Locations and Transportation Plan
Facilities to Open:
Facility 1 is opened.
Facility 2 is opened.

Transportation Plan:
Transport 80.0 units from Facility 1 to Customer 1.
Transport 10.0 units from Facility 1 to Customer 3.
Transport 60.0 units from Facility 1 to Customer 4.
Transport 70.0 units from Facility 2 to Customer 2.
Transport 80.0 units from Facility 2 to Customer 3.


In [34]:
# Function to update DataFrame with model results
def update_dataframe_with_model_results(df, model):
    # Extract values of decision variables
    x_values = model.x.extract_values()
    y_values = model.y.extract_values()

    # Update DataFrame to reflect facility openings and transportation decisions
    df['facility_open'] = df['facility'].apply(lambda i: x_values[i])
    df['transport_amount'] = df.apply(lambda row: y_values[(row['facility'], row['customer'])], axis=1)
    
    return df

In [38]:
updated_df = update_dataframe_with_model_results(df, model)

In [74]:
plot_facility_location(updated_df, plot_cost_edges=True, plot_transport_edges=True)


In [42]:
df

,facility,customer,fixed_cost,capacity,demand,transport_cost,facility_open,transport_amount
0,1,1,100,150,80,10,1.0,80.0
1,1,2,100,150,70,20,1.0,0.0
2,1,3,100,150,90,30,1.0,10.0
3,1,4,100,150,60,40,1.0,60.0
4,2,1,200,200,80,15,1.0,0.0
5,2,2,200,200,70,25,1.0,70.0
6,2,3,200,200,90,35,1.0,80.0
7,2,4,200,200,60,45,1.0,0.0
8,3,1,300,300,80,20,-0.0,0.0
9,3,2,300,300,70,30,-0.0,0.0


In [45]:
df

,facility,customer,fixed_cost,capacity,demand,transport_cost
0,1,1,100,150,80,10
1,1,2,100,150,70,20
2,1,3,100,150,90,30
3,1,4,100,150,60,40
4,2,1,200,200,80,15
5,2,2,200,200,70,25
6,2,3,200,200,90,35
7,2,4,200,200,60,45
8,3,1,300,300,80,20
9,3,2,300,300,70,30


### Appendix: **Check model components by `.display`**
> If data as `dictionary` is needed for further analysis. Refer to the usage of `.get_values()` for decision variables and `.extract_values()` for parameters in the Thesis `Model Implementation` Appendix Chapter.

In [23]:
# Enumerate names of components in Pyomo
compo_categories = [pyo.Var, pyo.Param, pyo.Expression, pyo.Objective, pyo.Constraint]
compo_category_name_list = ["Variable", "Param", "Expression", "Objective", "Constraint"]
compo_dictionary = {comp_cat_name: list() for comp_cat_name in compo_category_name_list}
for compo_cat_name, compo_cat in zip(compo_category_name_list, compo_categories):
    for compo in model.component_objects(compo_cat, active=True):
        compo_dictionary[compo_cat_name].append(str(compo))
#         print(f"{compo_cat_name}:", v)  

In [24]:
for key, value in compo_dictionary.items():
    print(f"{key}: {value}")

Variable: ['x', 'y']
Param: ['f', 'M', 'd', 'c']
Expression: []
Objective: ['objective']
Constraint: ['demand_constraint', 'capacity_constraint']


In [ ]:
model.x.display()

x : Size=3, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Binary
      2 :     0 :   1.0 :     1 : False : False : Binary
      3 :     0 :  -0.0 :     1 : False : False : Binary


In [25]:
model.y.display()

y : Size=12, Index=y_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :  80.0 :  None : False : False : NonNegativeReals
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (1, 3) :     0 :  10.0 :  None : False : False : NonNegativeReals
    (1, 4) :     0 :  60.0 :  None : False : False : NonNegativeReals
    (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (2, 2) :     0 :  70.0 :  None : False : False : NonNegativeReals
    (2, 3) :     0 :  80.0 :  None : False : False : NonNegativeReals
    (2, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
    (3, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals


In [32]:
model.y.extract_values()

{(1, 1): 80.0,
 (1, 2): 0.0,
 (1, 3): 10.0,
 (1, 4): 60.0,
 (2, 1): 0.0,
 (2, 2): 70.0,
 (2, 3): 80.0,
 (2, 4): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0}

In [27]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 8350.0


In [28]:
model.demand_constraint.display()

demand_constraint : Size=4
    Key : Lower : Body : Upper
      1 :  80.0 : 80.0 :  80.0
      2 :  70.0 : 70.0 :  70.0
      3 :  90.0 : 90.0 :  90.0
      4 :  60.0 : 60.0 :  60.0


In [29]:
model.capacity_constraint.display()

capacity_constraint : Size=3
    Key : Lower : Body  : Upper
      1 :  None :   0.0 :   0.0
      2 :  None : -50.0 :   0.0
      3 :  None :   0.0 :   0.0
